In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import zhon
import numpy as np
import re
import random
import pandas as pd
import json
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import set_seed
from transformers import BertTokenizerFast
from transformers import TFAutoModelForMaskedLM
from transformers import BertForPreTraining, BertForMaskedLM, BertConfig
import transformers
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
import argparse, torch, datasets, ast, operator, gc, os
import torch.nn as nn
import json
import csv
from math import ceil

In [12]:
parser = argparse.ArgumentParser(description='search for best template according to dev set')
parser.add_argument('--max_len', default=512, type=int, help="max sequence length")
parser.add_argument('--model', default='/home/lr/h1k/KNN-BERT-main/csi_sequetial_fine_tuning//MLM_task_tuned_models_for _datasets_PPL/MLM_tuned_roberta_large_JY_te+JY_Tr+CSI_tr/', type=str, help="pretrained model")
#parser.add_argument('--model', default='./chinese_roberta_wwm_large_ext_pytorch/')
parser.add_argument('--result_output_dir', default='/home/lr/h1k/KNN-BERT-main/csi_sequetial_fine_tuning/Tune_on_CSI/csi_output/ppl_results/', type=str, help="output directory")
parser.add_argument('--tokenizer', default='/home/lr/h1k/KNN-BERT-main/csi_sequetial_fine_tuning/MLM_task_tuned_models_for _datasets_PPL/MLM_tuned_roberta_large_JY_te+JY_Tr+CSI_tr/', type=str, help="tokenizer")
args = parser.parse_args(args=[])

# 加载模型和分词器
model = BertForMaskedLM.from_pretrained(args.model)
tokenizer = BertTokenizerFast.from_pretrained(f'{args.tokenizer}')
device = "cuda" if torch.cuda.is_available() else "cpu"

#if torch.cuda.device_count() > 1:
#model = torch.nn.DataParallel(model)
model.to(device)


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, 

In [13]:
with open("dialogue.json") as f:
    data=json.load(f)

In [14]:
type(data)

dict

In [28]:
len(data.keys())-1026-548-346

2222

In [32]:
data["4"][0]


{'speaker': '左正鵬',
 'utterance': '伯母、您好，我是你麗華的同學呀！',
 'listener': [{'name': '徐母', 'relation': 'son/daughter-in-law'}],
 'emotion': 'happiness'}

In [25]:
l_d=dict()
for i in set(length):
    l_d[i]=length.count(i)

In [26]:
l_d

{2: 1026,
 3: 548,
 4: 448,
 5: 346,
 6: 303,
 7: 239,
 8: 182,
 9: 179,
 10: 140,
 11: 128,
 12: 137,
 13: 87,
 14: 91,
 15: 69,
 16: 54,
 17: 53,
 18: 40,
 19: 45,
 20: 27}

In [76]:
def process(path_from,path_to, vocab_file,vocab_bond):
    
    with open(path_from) as f:
        dj=json.load(f)
    
    vf=open(vocab_file) 
    vocab= vf.readlines()
    vocab=[i.strip('\n') for i in vocab][vocab_bond[0]:vocab_bond[1]]
        
    name=path_from.split('/')[-1].strip('.csv')
    data= context_window(dj, vocab,name)
    
    with open(path_to, 'w') as f:
        json.dump(data, f, ensure_ascii=False,indent=2)

In [77]:
def examine_length(index_top, index_bottom, scene):
    ref=''
    utter=''
    for i in range(index_top, index_bottom):
        ref+=scene[i]['speaker']+scene[i]['utterance']
        utter+= scene[i]['utterance']
        window_num = ceil(len(ref)/512) 
    
    limit=512-len(utter)
    return window_num,limit

In [78]:
def context_window(segs, vocab, name):
    data_dict=dict()
    data_dict["data"]=list()
    data_dict["version"]="v1.0",
    
    i_scene=0
    for scene in segs.values(): 
        if len(scene)<=5:
            continue
        
        window_num, s_limit = examine_length(0,len(scene),scene) 

        if window_num==1:
            utterances=[u['utterance'] for u in scene]
            speakers=[u['speaker'] for u in scene]
            masked_sets=get_masked_sets(speakers)
            for m_speakers in masked_sets:
                paragraphs=lines_to_question(speakers,m_speakers, utterances, vocab, i_scene, name)
                data_dict["data"].append({"paragraphs":paragraphs})
            i_scene +=1

        else:
            norm_window_size=len(scene)//window_num
            pointer=0
            while pointer < len(scene)-5:
                pre_size=random.randint(min(6,norm_window_size-2),norm_window_size+1)
                window_size=min(pre_size, len(scene)-pointer)

                if not examine_length(pointer, pointer+window_size, scene)[0]==1:
                    norm_window_size-=1
                    continue
                utterances=[u['utterance'] for u in scene[pointer:pointer+window_size]]
                speakers=[u['speaker'] for u in scene[pointer:pointer+window_size]]
                masked_sets=get_masked_sets(speakers)

                for m_speakers in masked_sets:
                    paragraphs=lines_to_question(speakers,m_speakers, utterances, vocab, i_scene, name)
                    data_dict["data"].append({"paragraphs":paragraphs})
                i_scene +=1
                pointer += window_size
                
        print(i_scene)
        print("diag len",len(scene), "aug num",len(masked_sets))
    return data_dict

In [79]:
def noised_speaker(NAME):
    name=NAME
    while name==NAME or set(name)==1:
        if random.random()<0.4:
            mask=np.random.random(size = (len(name)))<0.5
            name=''.join([name[i] if mask[i]==True else '□' for i in range(len(name))])
        else:
            name=''
    return name

In [80]:
def structure_utter(SPEAKER,UTTER,T):
    R1=(random.random()<T)*(SPEAKER!="")
    R2=random.random()<T
    return R1*'：' + R2*'“' + UTTER + R2*'”'

In [81]:
def get_masked_sets(s, aug_time=6):

    #mask speakers
    speakers=s.copy()
    speaker_pointer=dict()
    for i in range(len(speakers)):
        if speakers[i] in speaker_pointer.keys():
            speaker_pointer[speakers[i]].append(i)
        else:
            speaker_pointer[speakers[i]]=list()
            speaker_pointer[speakers[i]].append(i)
    
    random_chance=3
    unmask_pos=list()
    while random_chance>=0 and len(unmask_pos)<aug_time and len(unmask_pos)<=len(s)//3:
        res=[random.choice(l) for l in speaker_pointer.values()]
        if not res in unmask_pos:
            unmask_pos.append(res)
        else:
            random_chance-=1
            
    masked_speakers=list()        
    for pos in unmask_pos:
        this_list=[noised_speaker(speakers[i]) if not i in pos else speakers[i] for i in range(len(speakers))]
        masked_speakers.append(this_list)
        
    return masked_speakers

In [82]:
def lines_to_question(s,ms,u,v, index, name):
    para=[{'id':name+"_"+str(index),'context':'','qas':[]}]
    context, back, answer= concat_with_noise(set(s),ms,u,v)
    para[-1]["context"]=context
    
    count=1
    qas=list()

    for i in range(len(u)+back):
        question={'question':u[i],'id':name+"_"+str(index)+"_"+str(count),'answers':[{"answer_start":answer[s[i]],"text":s[i]}]}
        count+=1
        qas.append(question)
        
    para[-1]["qas"]=qas
    return para

In [83]:
def concat_with_noise(speaker_set, ms, u, v, cloze_range=3, threshold1=0.5):
    mspeakers=ms.copy()

    cont, trace_back=cloze(u, mspeakers, cloze_range, threshold1)
    speaker_answer=dict()
    
    for speaker in speaker_set:
        speaker_answer[speaker]=cont.find(speaker)
 
    return cont, trace_back, speaker_answer

In [84]:
def cloze(utter, m_speaker, cloze_range, threshold1):
    sequence=''
    pre_sequence=''
    back=0
    for i in range(len(m_speaker)):
        ranint1=random.randint(0,cloze_range)
        ranint2=random.randint(0,cloze_range)
        random_threshold1= threshold1
        pre_sequence+=m_speaker[i]+(m_speaker[i]!="")*ranint1*tokenizer.mask_token + structure_utter(m_speaker[i],utter[i]+ tokenizer.mask_token*ranint2, random_threshold1) + tokenizer.sep_token
        if len(pre_sequence)<512:
            sequence=pre_sequence
        else:
            back=i-len(m_speaker)
            break


    input = tokenizer.encode(sequence, return_tensors="pt")
    mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
    token_logits = model(input.cuda()).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_1_tokens = torch.topk(mask_token_logits, 1, dim=1).indices.squeeze(1)
    
    for i in range(len(mask_token_index)):
        sequence=sequence.replace(tokenizer.mask_token, tokenizer.decode(top_1_tokens[i]),1)
    sequence=sequence.replace(tokenizer.sep_token,'')
    
    return sequence, back

In [ ]:
from_path='./dialogue.json'
to_path='./CRECIL_train.json'
vocab_file='/home/lr/h1k/KNN-BERT-main/csi_sequetial_fine_tuning/chinese_roberta_wwm_large_ext_pytorch/vocab.txt'
bond_vocab=[671,7992]
process(from_path,to_path,vocab_file,bond_vocab)

1
diag len 6 aug num 3
2
diag len 9 aug num 4
3
diag len 8 aug num 3
9
diag len 18 aug num 2
11
diag len 15 aug num 3
12
diag len 7 aug num 3
14
diag len 16 aug num 3
15
diag len 7 aug num 3
16
diag len 9 aug num 4
17
diag len 7 aug num 3
18
diag len 6 aug num 3
19
diag len 7 aug num 3
21
diag len 14 aug num 2
22
diag len 6 aug num 3
23
diag len 6 aug num 3
24
diag len 6 aug num 3
25
diag len 7 aug num 3
27
diag len 18 aug num 3
28
diag len 10 aug num 4
29
diag len 13 aug num 5
30
diag len 8 aug num 3
32
diag len 11 aug num 2
33
diag len 7 aug num 3
34
diag len 6 aug num 3
36
diag len 15 aug num 2
37
diag len 6 aug num 3
38
diag len 7 aug num 3
40
diag len 20 aug num 4
42
diag len 17 aug num 3
44
diag len 17 aug num 3
45
diag len 6 aug num 3
46
diag len 7 aug num 3
47
diag len 15 aug num 6
48
diag len 10 aug num 4
49
diag len 13 aug num 5
51
diag len 16 aug num 3
52
diag len 8 aug num 3
53
diag len 11 aug num 4
54
diag len 6 aug num 3
55
diag len 10 aug num 4
56
diag len 9 aug num 4
58

368
diag len 8 aug num 3
369
diag len 17 aug num 6
370
diag len 7 aug num 3
371
diag len 9 aug num 4
372
diag len 8 aug num 3
373
diag len 10 aug num 4
374
diag len 6 aug num 3
375
diag len 14 aug num 5
376
diag len 8 aug num 3
377
diag len 12 aug num 5
378
diag len 6 aug num 3
379
diag len 6 aug num 3
380
diag len 15 aug num 6
381
diag len 16 aug num 6
382
diag len 12 aug num 5
383
diag len 8 aug num 3
384
diag len 8 aug num 3
385
diag len 6 aug num 3
386
diag len 18 aug num 6
387
diag len 10 aug num 4
388
diag len 8 aug num 3
389
diag len 9 aug num 4
390
diag len 6 aug num 3
391
diag len 10 aug num 4
392
diag len 7 aug num 2
393
diag len 7 aug num 3
394
diag len 9 aug num 4
395
diag len 10 aug num 4
396
diag len 7 aug num 3
397
diag len 14 aug num 5
398
diag len 6 aug num 3
399
diag len 8 aug num 3
400
diag len 7 aug num 3
401
diag len 13 aug num 5
402
diag len 7 aug num 3
403
diag len 6 aug num 3
404
diag len 6 aug num 3
405
diag len 9 aug num 4
406
diag len 13 aug num 5
407
diag le

700
diag len 9 aug num 4
701
diag len 6 aug num 3
702
diag len 12 aug num 5
703
diag len 7 aug num 3
704
diag len 13 aug num 5
705
diag len 8 aug num 3
706
diag len 11 aug num 4
707
diag len 6 aug num 3
708
diag len 8 aug num 3
709
diag len 8 aug num 3
710
diag len 13 aug num 5
711
diag len 11 aug num 4
712
diag len 13 aug num 5
713
diag len 8 aug num 3
714
diag len 19 aug num 6
716
diag len 17 aug num 4
717
diag len 19 aug num 6
718
diag len 10 aug num 2
719
diag len 11 aug num 4
720
diag len 11 aug num 4
722
diag len 19 aug num 3
723
diag len 12 aug num 5
724
diag len 13 aug num 5
725
diag len 15 aug num 6
726
diag len 20 aug num 6
727
diag len 9 aug num 4
728
diag len 8 aug num 3
729
diag len 6 aug num 3
730
diag len 10 aug num 4
731
diag len 10 aug num 4
733
diag len 16 aug num 3
734
diag len 17 aug num 6
735
diag len 6 aug num 3
737
diag len 18 aug num 3
738
diag len 7 aug num 3
739
diag len 7 aug num 3
740
diag len 10 aug num 4
742
diag len 17 aug num 2
743
diag len 16 aug num 6


1056
diag len 10 aug num 4
1057
diag len 12 aug num 3
1058
diag len 10 aug num 4
1059
diag len 6 aug num 3
1060
diag len 9 aug num 4
1061
diag len 6 aug num 3
1062
diag len 11 aug num 4
1063
diag len 9 aug num 4
1065
diag len 13 aug num 1
1066
diag len 8 aug num 3
1067
diag len 6 aug num 3
1068
diag len 12 aug num 5
1069
diag len 7 aug num 3
1070
diag len 9 aug num 4
1071
diag len 6 aug num 3
1072
diag len 6 aug num 3
1073
diag len 10 aug num 4
1074
diag len 12 aug num 5
1075
diag len 14 aug num 5
1076
diag len 10 aug num 4
1078
diag len 13 aug num 2
1079
diag len 12 aug num 5
1080
diag len 9 aug num 4
1082
diag len 12 aug num 3
1084
diag len 15 aug num 3
1085
diag len 11 aug num 4
1087
diag len 16 aug num 3
1088
diag len 6 aug num 3
1089
diag len 7 aug num 3
1090
diag len 7 aug num 3
1092
diag len 20 aug num 3
1094
diag len 13 aug num 3
1095
diag len 8 aug num 3
1096
diag len 11 aug num 4
1097
diag len 6 aug num 3
1098
diag len 6 aug num 3
1099
diag len 6 aug num 3
1100
diag len 14 au

In [ ]:
min(1,3)

In [121]:
"qwedfwdcsdasdcq"+"q"

'qwedfwdcsdasdcqq'

In [134]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Jan 12 23:22:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:81:00.0 Off |                  Off |
| 39%   66C    P2    98W / 300W |  35720MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:82:00.0 Off |                  Off |
| 63%   